# For analyzing the 2020 Census Data for Michigan and the Detroit area.
Notebook to assist in the process of redistricting for Kent County. Right now the data is going to be based on the Census ACS Survery. It is still unclear if the Offical Census Data will be available before the process needs to be completed.

In [ ]:
import os
from shutil import copy2
import pandas as pd
import numpy as np
from datetime import datetime as dt
from pathlib import Path
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor
import requests
from dotenv import find_dotenv, load_dotenv
from time import time

In [ ]:
now = dt.now()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'

In [ ]:
#Create Folders for Census Data
cFolder = [f for f in lPath if f.name == 'Processing'][0]
cProcessing = cFolder / 'Census' / f'{dStr}'
if cProcessing.exists() == True:
    print(f'{cProcessing} already exist.')
else:
    cProcessing.mkdir(parents=True)
    print(f'Created {cProcessing}.')

cFR = [f for f in lPath if f.name == 'Review'][0]
cReview = cFR / 'Census' / f'{dStr}'
if cReview.exists() == True:
    print(f'{cReview} already exist')
else:
    cReview.mkdir(parents=True)
    print(f'Created {cReview}')

In [ ]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / cFolder / f'Data_{mStr}.gdb'
# dsA = gisPath / pFolder / f'{outGDB}.gdb'
locGDB = outGDB / 'Census2020'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{cFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

In [ ]:
load_dotenv(find_dotenv())

gis_user = os.getenv('ESRI_USERNAME')
gis_pass = os.getenv('ESRI_PASSWORD')
gis_url = os.getenv('PORTAL_URL')
agol_user = os.getenv('AGOL_USERNAME')
agol_pass = os.getenv('AGOL_PASSWORD')
agol_url = os.getenv('AGOL_SITE')

from arcgis import GIS

gisE = GIS(url=gis_url, username=gis_user, password=gis_pass)

gisA = GIS(url=agol_url, username=agol_user, password=agol_pass)

gis = GIS()

### Get Census Data for Kent County from ACS 5 Year Survey. Data is going to be at the block group level instead of the block level. 

The field names will need to be changed to match the field names in the Census 2020 data.

In [ ]:
census_key = os.getenv('CENSUS_KEY')

cvar = {
    'GEO_ID':'GEOID',
    'B01001_001E':'est_total',
    'B01001_001EA':'est_total_anno',
    'B01001_001M':'moe',
    'B01001_001MA':'moe_anno',
    'NAME':'name'
}

# Request parameters and grab all the data associated with Kent County (does not contain spatial information)
payload = {
    'get' : ','.join(list(cvar.keys())),
    'for' : 'block group:*',
    'in' : ['state:10', 'county:001'],
    'key' : census_key
}

census_url = f'https://api.census.gov/data/2019/acs/acs5'

res = requests.get(census_url, params=payload)

if res.status_code == 200:
    data = res.json()
    c_df = pd.DataFrame(data[1:], columns=data[0])
    c_df.rename(columns=cvar, inplace=True)

In [ ]:
#change the GEOID to work with merge/join later with spatial data
c_df.loc[:, 'GEOID'] = c_df.GEOID.str[9:]

#drop the columns that are not necessary
c_df.drop(columns=['state', 'county', 'tract', 'block group'], inplace=True)
c_df.sort_values(by='GEOID', inplace=True, ignore_index=True)

#change the columns data types to match the data contained
census_dtypes = {
    'GEOID':'string',
    'name':'string',
    'est_total':'int32',
    'est_total_anno':'string',
    'moe':'int32',
    'moe_anno':'string'
}

c_df = c_df[list(census_dtypes.keys())].astype(census_dtypes)

c_df.fillna('', inplace=True)

#export table to local GDB
tabName = locGDB.parent / 'CensusACS_2019'
c_df.spatial.to_table(str(tabName))
print(f'{tabName.name} has been created')

### Get Census Block Group Geographic Data from Census REST Server. For this example, I am using ArcGIS API for Python, but you can use Open Source libraries like Geopandas.

In [ ]:
#Grab the spatial data from Census REST Server and query by Kent County
outCen = locGDB / '2019_Blocks'
flc = FeatureLayerCollection('https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer') #This is getting the census tracts for 2020

expr = 'STATE = 10 AND COUNTY = 01'

flList = flc.layers

cbg = flList[0].query(out_sr=26957, where=expr, as_df=True)
cbg.spatial.to_featureclass(f'{gis}', sanitize_columns=False)

In [ ]:
expr = 'STATE = 10 AND COUNTY = 01'
for x in flList:
    c = x.properties.description.split(',')[-1]
    # print(c)
    if '2020' in c:
        print(c)
        # d = x.query(where=expr, out_sr=26957, as_df=True)

In [ ]:
# flc = FeatureLayerCollection('https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer').layers
# a = flc[10].properties
# .description.split(',')[-1]

# blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/2/query?'

# blocks_params = {
#     'where' : 'STATE = 10 and COUNTY = 001',
#     'outFields' : '*',
#     'f' : 'GeoJSON'
# }

# blocks = requests.get(blocks_url, blocks_params)

# gj = blocks.json()

# arcpy.JSONToFeatures_conversion(gj, f'{gis / "census2"}', 'POLYGON')


expr = 'STATE = 10 AND COUNTY = 01'
# d = flc[0].query(where=expr, out_sr=4326, as_df=True)
# d.spatial.to_featureclass(location=f'{netDir / "Census"}', sanitized_columns=False)
if '2020' in flc[0].properties.description.split(',')[-1]:
    d = flc[0].query(where=expr, out_sr=4326)
    d.save(f'{outGDB}', f'{flc[0].properties.name}')
    print(f'Finishing Copying {flc[0].properties.name}')
for f in flc:
    c = f.properties.description.split(',')[-1]
    if '2020' in c:
        d = f.query(where=expr, out_sr=4326, as_df=True)
        # d.save(f'{outGDB}', )
        d.spatial.to_featureclass(location=f'{outGDB / outGDB.name}', sanitize_columns=False)
        # d.save(f'{outGDB}', f'{f.properties.name}')
        # arcpy.FeatureClassToFeatureClass_conversion(d, f'{outGDB}', f'{f.properties.name}')
        print(f'Copied {f}')

print(flc[2].properties.name)


cbg_df = FeatureLayer(flc).query(out_sr= 4326, where=expr, out_fields='GEOID',as_df=True)

cbg_df.sort_values(by='GEOID', inplace=True, ignore_index=True)

<b> Method using GeoPandas instead of ArcGIS Python API. </b>

In [ ]:
try:
    import geopandas as gpd

    blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/2/query?'

    blocks_params = {
        'where' : 'STATE = 10 and COUNTY = 001',
        'outFields' : '*',
        'f' : 'GeoJSON'
    }

    blocks = requests.get(blocks_url, blocks_params)

    cbg_gdf = gpd.read_file(blocks.text)
    cbg_gdf.sort_values(by='GEOID', inplace=True, ignore_index=True)

except ImportError:
    print("Need to install geopandas in the current environment.")

In [ ]:
import geopandas as gpd
import requests

blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/8/query?'

blocks_params = {
    'where' : 'STATE = 10 and COUNTY = 001',
    'outFields' : '*',
    'f' : 'JSON'
}

blocks = requests.get(blocks_url, blocks_params)

cbg_gdf = gpd.read_file(blocks.text)

cbg_gdf.sort_values(by='GEOID', inplace=True, ignore_index=True)

In [ ]:
cbg_gdf.to_file(f'{gis.parent / "gdCensus.shp"}', index=False)

Export to ESRIDB.

In [ ]:
kc_df = cbg_df[['SHAPE']].join(c_df)

In [ ]:
netDir = Path(r'C:\Users\MKinnaman\AppData\Roaming\ESRI\Desktop10.5\ArcCatalog')
netDB = netDir / 'GISPro' / 'SDE Connections'
esriDB = netDir / 'GISAdmin.sde' / 'PROD.GISADMIN.Census_ACS_2019'
outFC = esriDB / 'TotalPop_2019'
# kc_df.spatial.to_featureclass(location=f'{outFC}', sanitize_columns=False)

### Merge or join the Census API Data to Census Spatial Data 

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

cbg_gdf.plot()

In [ ]:
# Define figure
fig, ax = plt.subplots(1,1)

# Plot dataframe to figure
div = make_axes_locatable(ax)
cax = div.append_axes("bottom", size='5%', pad=0.1)
kc_gdf.plot(column='est_total', legend=True, ax=ax, legend_kwds={'orientation':'horizontal'}, cax=cax, edgecolor='black', linewidth=0.4, cmap='BrBG')

# Adjust figure details
ax.set_title('Estimated Population Change from 2010-2014 to 2015-2019')
ax.axes.xaxis.set_visible(False); ax.axes.yaxis.set_visible(False)
ax.set_frame_on(False)
fig.set_figwidth(10); fig.set_figheight(10)

plt.show()

## Get Geographic Data from Census REST Server

Get Kent County Census Data using Arcgis Python API

In [ ]:
netDir = Path(r'C:\Users\MKinnaman\AppData\Roaming\ESRI\Desktop10.5\ArcCatalog')
esriDB = netDir / 'GISAdmin.sde'

In [ ]:
iE = esriDB / 'PROD.GISADMIN.Census2010'
sr = arcpy.Describe(f'{iE}').spatialReference
outGDB = esriDB / 'PROD.GISADMIN.Census2020'
# dsA = Path(gisPath / nGDB / f'{outGDB}.gdb')
# locGDB = Path(dsA / f'Daily_{dStr}')
# if arc.Exists(outGDB):
#     print("GDB already exists.")
# else:
#     arc.CreateFileGDB_management(nGDB, outGDB)
#     print(f'Created File GeoDatabase at {dsA}')

# time.sleep(5)
if arcpy.Exists(f'{outGDB}'):
    print(f'{outGDB.name} already exists')
else:
    arcpy.CreateFeatureDataset_management(f'{outGDB.parent}', f'{outGDB.name}', sr)
    print(f'{outGDB.name} Dataset has been created')

In [ ]:
home = Path.home()
gis = home / 'GIS' / 'Processing'

# flc = FeatureLayerCollection('https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer').layers
# a = flc[10].properties
# .description.split(',')[-1]

blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/2/query?'

blocks_params = {
    'where' : 'STATE = 10 and COUNTY = 001',
    'outFields' : '*',
    'f' : 'GeoJSON'
}

blocks = requests.get(blocks_url, blocks_params)

gj = blocks.json()

arcpy.JSONToFeatures_conversion(gj, f'{gis / "census2"}', 'POLYGON')


expr = 'STATE = 10 AND COUNTY = 01'
d = flc[0].query(where=expr, out_sr=4326, as_df=True)
d.spatial.to_featureclass(location=f'{netDir / "Census"}', sanitized_columns=False)
if '2020' in flc[0].properties.description.split(',')[-1]:
    d = flc[0].query(where=expr, out_sr=4326)
    d.save(f'{outGDB}', f'{flc[0].properties.name}')
    print(f'Finishing Copying {flc[0].properties.name}')
for f in flc:
    c = f.properties.description.split(',')[-1]
    if '2020' in c:
        d = f.query(where=expr, out_sr=4326, as_df=True)
        # d.save(f'{outGDB}', )
        d.spatial.to_featureclass(location=f'{outGDB / outGDB.name}', sanitize_columns=False)
        # d.save(f'{outGDB}', f'{f.properties.name}')
        # arcpy.FeatureClassToFeatureClass_conversion(d, f'{outGDB}', f'{f.properties.name}')
        print(f'Copied {f}')

print(flc[2].properties.name)


cbg_df = FeatureLayer(fl).query(out_sr= 4326, where=expr, out_fields='GEOID',as_df=True)

cbg_df.sort_values(by='GEOID', inplace=True, ignore_index=True)

Export Kent County Census Blocks using Geopandas

In [ ]:
    blocks_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/2/query?'

    blocks_params = {
        'where' : 'STATE = 10 and COUNTY = 001',
        'outFields' : '*',
        'f' : 'GeoJSON'
    }

    blocks = requests.get(blocks_url, blocks_params)


    cbg_gdf = gpd.read_file(blocks.text)



    cbg_gdf.sort_values(by='GEOID', inplace=True, ignore_index=True)

    home = Path.home()
    gis = home / 'GIS' / 'Processing'
    cbg_gdf.to_file(f'{gis / "census.shp"}')
